# Week 3: Divergence and vorticity
### MAQ - 32806, Chiel van Heerwaarden & Imme Benedict, 2016
In this assignment you will investigate whether the large-scale flow is in geostrophic and thermal wind balance. For this, you will use again the ECMWF ERA-Interim data from the 1st of January 2016 0:00. Note the new `get_pressure_index` function. This makes it easier to retrieve data at a certain pressure level.
___
First, we load the required packages.

In [ ]:
import numpy as np              # Numpy is the fundamental package for scientific computing in Python.
import netCDF4 as nc            # NetCDF is the data format of the meteorological data that we use.
import matplotlib.pyplot as pl  # Matplotlib is a scientific plotting package.
from mpl_toolkits.basemap import Basemap # Import the map plotting interface.
from ipywidgets import interact
import warnings
warnings.filterwarnings('ignore')

# The statement below enforces the plots to be put into this notebook, instead of in their own windows.
%matplotlib inline
pl.rcParams.update({'font.size': 11})          # Set the standard font size of the plots to 11pt.
pl.rcParams.update({'figure.figsize': [13,6]}) # Set the standard figure size.

___
### Understanding divergence, vorticity and deformation

Before studying weather data, you will first study the two-dimensional flow patterns associated with divergence $\dfrac{\partial u}{\partial x} + \dfrac{\partial v}{\partial y}$, vorticity $\dfrac{\partial v}{\partial x} - \dfrac{\partial u}{\partial y}$, and the two components of deformation $\dfrac{\partial u}{\partial x} - \dfrac{\partial v}{\partial y}$ and $\dfrac{\partial v}{\partial x} + \dfrac{\partial u}{\partial y}$. Every infinitisimal velocity field can be described as a linear combination of these four. See Holton section 1.5 for a full explanation.

In [ ]:
def plot_streamlines(div=0, zeta=0, d1=0, d2=0):
    n = 65
    x = np.linspace(-1., 1., n)
    y = np.linspace(-1., 1., n)
    u = 0.5*(div +d1)*x[np.newaxis, :] + 0.5*(d2 -zeta)*y[:,np.newaxis]
    v = 0.5*(zeta+d2)*x[np.newaxis, :] + 0.5*(div-d1  )*y[:,np.newaxis]
    U = (u**2 + v**2)**.5

    pl.figure(figsize=(6,4))
    pl.subplot(111, aspect='equal')
    pl.contourf(x, y, U, 21)
    pl.colorbar()
    pl.streamplot(x, y, u, v, color='k', arrowsize=2.)
    pl.xlabel('x')
    pl.ylabel('y')
    pl.show()
    
interact(plot_streamlines,
         div=(-1,1,0.1), zeta=(-1,1,0.1), d1=(-1,1,0.1), d2=(-1,1,0.1));

___
### Setting up a Python Basemap and read out 3D fields
With the code below, you initialize the world map with the desired coordinates.

In [ ]:
# We define a map of the northern hemisphere that is stored in the m object.
lon_start =  - 70.
lon_end   =    30.
lat_start =    30.
lat_end   =    61.
m = Basemap(llcrnrlon=lon_start,
            llcrnrlat=lat_start,
            urcrnrlon=lon_end,
            urcrnrlat=lat_end,
            projection='mill', resolution='l')

___
Now, you load the data and read out the desired variables from a NetCDF file from the ECMWF ERA-Interim data archive.

In [ ]:
nc_file = nc.Dataset("data/era_data.nc", "r")
lat = nc_file.variables["latitude"][:]
lon = nc_file.variables["longitude"][:]
p = nc_file.variables["level"][:]*100.
t = 0
nroll = lon.size//2
lon = np.roll(lon, nroll)
lon = np.where(lon>=180., lon-360., lon)

u     = np.roll(nc_file.variables["u"][t,:,:,:], nroll, -1)
v     = np.roll(nc_file.variables["v"][t,:,:,:], nroll, -1)
omega = np.roll(nc_file.variables["w"][t,:,:,:], nroll, -1)
Phi   = np.roll(nc_file.variables["z"][t,:,:,:], nroll, -1)
T     = np.roll(nc_file.variables["t"][t,:,:,:], nroll, -1)
slp   = np.roll(nc_file.variables["msl"][t,:,:], nroll, -1)
pr    = np.roll(nc_file.variables["lsp"][t,:,:], nroll, -1)\
      + np.roll(nc_file.variables["cp" ][t,:,:], nroll, -1)

nc_file.close()

lons, lats = m(*np.meshgrid(lon, lat))
lonrad, latrad = np.meshgrid(np.pi/180.*lon, np.pi/180.*lat)

def get_pressure_index(p_plot):
    return abs(p-p_plot).argmin()

nq = 3
qscale = 2000.

___
### Plotting the temperature, geopotential and wind vectors
As a starting point you find below the temperature and geopotential at 500 hPa.

In [ ]:
n = get_pressure_index(50000.)
quiver = False # Set this flag to True if you want to enable the wind arrows.

pl.figure()
m.contourf(lons, lats, T[n,:,:], 31, cmap=pl.cm.jet)
m.colorbar()
if (quiver):
    qu = m.quiver(lons[::nq,::nq], lats[::nq,::nq],
                  u[n,::nq,::nq], v[n,::nq, ::nq],
                  pivot='mid', width=1.2e-3,
                  scale=qscale)

cz = m.contour(lons, lats, Phi[n,:,:], 15, colors='k', linewidths=1.5)
pl.clabel(cz, fontsize=10., fmt='%1.0f')

cp = m.contour(lons, lats, slp, 15, colors='k', linestyles=':', linewidths=1.)
pl.clabel(cp, fontsize=10., fmt='%1.0f')

m.drawcoastlines(color='#666666')
# draw parallels.
parallels = np.arange(-90.,90,15.)
m.drawparallels(parallels,labels=[1,0,0,0])
# draw meridians
meridians = np.arange(0.,360.,15.)
m.drawmeridians(meridians,labels=[0,0,0,1])
pl.title('T and $\Phi$ at p = {0:.0f} hPa and SLP'.format(p[n] / 100));
pl.tight_layout()

___
### Assignment
In this assignment you are going to analyze the structure of vorticity and divergence surrounding the cyclone at 48N, 20W. Before doing so, you will get familiar with divergence, vorticity and deformation.

#### A first glimpse on vorticity, divergence and deformation
Spatial variations in the velocity field can alter a moving air parcel in three possible ways. The variations can introduce *divergence*, *vorticity* and *deformation*. Use the applet at the top of the notebook and move the sliders to change the relative contribution of eacht of the three. The values in this applet are scaled and range between -1 and 1. The purpose of this exercise is to understand the patterns, not to understand its magnitudes. Make sure to read Holton section 1.5.

1. Explain the flow pattern for a flow with only divergence. What happens if you change the sign?
1. Explain the flow pattern for a flow with only vorticity. What happens if you change the sign?
1. Explain the flow pattern for a flow with only deformation. What happens if you change the sign?
1. Explain the flow pattern for a flow with divergence and vorticity. What happens if you change the relative magnitude of one of the two? Which combinations do you find in high and low pressure systems on the Northern hemisphere? Which not?
1. Combine vorticity and one of the two deformation components and put both at the same magnitude. What do you observe? How is rotation generated in such a flow pattern?

#### Vorticity on the weather map
In this exercise, focus your answers on the surroundings of the cyclone at 48N and 20W.
1. Calculate the relative vorticity $\zeta \equiv \dfrac{\partial v}{\partial x} - \dfrac{\partial u}{\partial y}$, and plot it at 850, 500 and 300 hPa. Explain the patterns and the differences you observe.
2. Calculate and plot the planetary vorticity $f$ at the 500 hPa level. How does it look at 300 hPa?
3. Plot the absolute vorticity $\eta \equiv \zeta + f$ at 300 hPa. What do you observe? Which of its two components dominates?

#### Divergence and vertical velocity on the weather map
In this exercise, focus your answers on the surroundings of the cyclone at 48N and 20W.
1. Calculate the horizontal divergence $\dfrac{\partial u}{\partial x} + \dfrac{\partial v}{\partial y}$ and plot it at 850, 500 and 300 hPa. Explain the patterns and the differences you observe.
2. Plot the vertical velocity $\omega$ at the 500 hPa level. What do you observe? How does it compare to the divergence plots?
3. Plot the precipitation `pr` and $\omega$ at the 500 hPa level in the same plot and discuss whether and why the two look similar.
4. Make a vertical cross section along latitude of 48N of the temperature and plot the wind vectors in there with the `m.quiver` function. TIP: multiply $\omega$ with -10 to be able to observe changes in the vertical velocity. Does the vertical profile corroborate your earlier explanations?
___
Work out the assignment below.
____

In [ ]:
a = 6.37e6
dvdx = 1./(a * np.cos(latrad)) * np.gradient(v, axis=2) / np.gradient(lonrad, axis=1)
dudy = 1./(a * np.cos(latrad)) * np.gradient(u*np.cos(latrad), axis=1) \
                               / np.gradient(latrad, axis=0)
zeta = dvdx - dudy

Omega = 7.292e-5
f = 2.*Omega*np.sin(latrad)

In [ ]:
zeta_range = np.linspace(-3e-4, 3e-4, 21)

# FIGURE 1
n = get_pressure_index(85000.)

pl.figure()
m.contourf(lons, lats, zeta[n,:,:], zeta_range, cmap=pl.cm.seismic, extend='both')
m.colorbar()

cz = m.contour(lons, lats, Phi[n,:,:], 15, colors='k', linewidths=1.5)
pl.clabel(cz, fontsize=10., fmt='%1.0f')

cp = m.contour(lons, lats, slp, 15, colors='k', linestyles=':', linewidths=1.)
pl.clabel(cp, fontsize=10., fmt='%1.0f')

m.drawcoastlines(color='#666666')
# draw parallels.
parallels = np.arange(-90.,90,15.)
m.drawparallels(parallels,labels=[1,0,0,0])
# draw meridians
meridians = np.arange(0.,360.,15.)
m.drawmeridians(meridians,labels=[0,0,0,1])
pl.title('$\zeta$ and $\Phi$ at p = {0:.0f} hPa'.format(p[n] / 100));
pl.tight_layout()

# FIGURE 2
n = get_pressure_index(50000.)

pl.figure()
m.contourf(lons, lats, zeta[n,:,:], zeta_range, cmap=pl.cm.seismic, extend='both')
m.colorbar()

cz = m.contour(lons, lats, Phi[n,:,:], 15, colors='k', linewidths=1.5)
pl.clabel(cz, fontsize=10., fmt='%1.0f')

cp = m.contour(lons, lats, slp, 15, colors='k', linestyles=':', linewidths=1.)
pl.clabel(cp, fontsize=10., fmt='%1.0f')

m.drawcoastlines(color='#666666')
# draw parallels.
parallels = np.arange(-90.,90,15.)
m.drawparallels(parallels,labels=[1,0,0,0])
# draw meridians
meridians = np.arange(0.,360.,15.)
m.drawmeridians(meridians,labels=[0,0,0,1])
pl.title('$\zeta$ and $\Phi$ at p = {0:.0f} hPa'.format(p[n] / 100));
pl.tight_layout()

# FIGURE 3
n = get_pressure_index(30000.)

pl.figure()
m.contourf(lons, lats, zeta[n,:,:], zeta_range, cmap=pl.cm.seismic, extend='both')
m.colorbar()

cz = m.contour(lons, lats, Phi[n,:,:], 15, colors='k', linewidths=1.5)
pl.clabel(cz, fontsize=10., fmt='%1.0f')

cp = m.contour(lons, lats, slp, 15, colors='k', linestyles=':', linewidths=1.)
pl.clabel(cp, fontsize=10., fmt='%1.0f')

m.drawcoastlines(color='#666666')
# draw parallels.
parallels = np.arange(-90.,90,15.)
m.drawparallels(parallels,labels=[1,0,0,0])
# draw meridians
meridians = np.arange(0.,360.,15.)
m.drawmeridians(meridians,labels=[0,0,0,1])
pl.title('$\zeta$ and $\Phi$ at p = {0:.0f} hPa'.format(p[n] / 100));
pl.tight_layout()

In [ ]:
# FIGURE 1
n = get_pressure_index(30000.)

pl.figure()
m.contourf(lons, lats, f, zeta_range, cmap=pl.cm.seismic, extend='both')
m.colorbar()

cz = m.contour(lons, lats, Phi[n,:,:], 15, colors='k', linewidths=1.5)
pl.clabel(cz, fontsize=10., fmt='%1.0f')

cp = m.contour(lons, lats, slp, 15, colors='k', linestyles=':', linewidths=1.)
pl.clabel(cp, fontsize=10., fmt='%1.0f')

m.drawcoastlines(color='#666666')
# draw parallels.
parallels = np.arange(-90.,90,15.)
m.drawparallels(parallels,labels=[1,0,0,0])
# draw meridians
meridians = np.arange(0.,360.,15.)
m.drawmeridians(meridians,labels=[0,0,0,1])
pl.title('$\zeta$ and $\Phi$ at p = {0:.0f} hPa'.format(p[n] / 100));
pl.tight_layout()

# FIGURE 2
pl.figure()
m.contourf(lons, lats, zeta[n,:,:] + f, zeta_range, cmap=pl.cm.seismic, extend='both')
m.colorbar()

cz = m.contour(lons, lats, Phi[n,:,:], 15, colors='k', linewidths=1.5)
pl.clabel(cz, fontsize=10., fmt='%1.0f')

cp = m.contour(lons, lats, slp, 15, colors='k', linestyles=':', linewidths=1.)
pl.clabel(cp, fontsize=10., fmt='%1.0f')

m.drawcoastlines(color='#666666')
# draw parallels.
parallels = np.arange(-90.,90,15.)
m.drawparallels(parallels,labels=[1,0,0,0])
# draw meridians
meridians = np.arange(0.,360.,15.)
m.drawmeridians(meridians,labels=[0,0,0,1])
pl.title('$\zeta$ and $\Phi$ at p = {0:.0f} hPa'.format(p[n] / 100));
pl.tight_layout()

In [ ]:
dudx = 1./(a * np.cos(latrad)) * np.gradient(u, axis=2) / np.gradient(lonrad, axis=1)
dvdy = 1./(a * np.cos(latrad)) * np.gradient(v*np.cos(latrad), axis=1) \
                               / np.gradient(latrad, axis=0)
divh = dudx + dvdy

In [ ]:
div_range = np.linspace(-7e-5, 7e-5, 21)

# FIGURE 1
n = get_pressure_index(85000.)

pl.figure()
m.contourf(lons, lats, divh[n,:,:], div_range, cmap=pl.cm.seismic, extend='both')
m.colorbar()

cz = m.contour(lons, lats, Phi[n,:,:], 15, colors='k', linewidths=1.5)
pl.clabel(cz, fontsize=10., fmt='%1.0f')

cp = m.contour(lons, lats, slp, 15, colors='k', linestyles=':', linewidths=1.)
pl.clabel(cp, fontsize=10., fmt='%1.0f')

m.drawcoastlines(color='#666666')
# draw parallels.
parallels = np.arange(-90.,90,15.)
m.drawparallels(parallels,labels=[1,0,0,0])
# draw meridians
meridians = np.arange(0.,360.,15.)
m.drawmeridians(meridians,labels=[0,0,0,1])
pl.title('div(V) at p = {0:.0f} hPa and $\Phi$ at p = {1:.0f} hPa'.format(p[n] / 100, p[n] / 100));
pl.tight_layout()

# FIGURE 2
n = get_pressure_index(50000.)

pl.figure()
m.contourf(lons, lats, divh[n,:,:], div_range, cmap=pl.cm.seismic, extend='both')
m.colorbar()

cz = m.contour(lons, lats, Phi[n,:,:], 15, colors='k', linewidths=1.5)
pl.clabel(cz, fontsize=10., fmt='%1.0f')

cp = m.contour(lons, lats, slp, 15, colors='k', linestyles=':', linewidths=1.)
pl.clabel(cp, fontsize=10., fmt='%1.0f')

m.drawcoastlines(color='#666666')
# draw parallels.
parallels = np.arange(-90.,90,15.)
m.drawparallels(parallels,labels=[1,0,0,0])
# draw meridians
meridians = np.arange(0.,360.,15.)
m.drawmeridians(meridians,labels=[0,0,0,1])
pl.title('div(V) at p = {0:.0f} hPa and $\Phi$ at p = {1:.0f} hPa'.format(p[n] / 100, p[n] / 100));
pl.tight_layout()

# FIGURE 3
n = get_pressure_index(30000.)

pl.figure()
m.contourf(lons, lats, divh[n,:,:], div_range, cmap=pl.cm.seismic, extend='both')
m.colorbar()

cz = m.contour(lons, lats, Phi[n,:,:], 15, colors='k', linewidths=1.5)
pl.clabel(cz, fontsize=10., fmt='%1.0f')

cp = m.contour(lons, lats, slp, 15, colors='k', linestyles=':', linewidths=1.)
pl.clabel(cp, fontsize=10., fmt='%1.0f')

m.drawcoastlines(color='#666666')
# draw parallels.
parallels = np.arange(-90.,90,15.)
m.drawparallels(parallels,labels=[1,0,0,0])
# draw meridians
meridians = np.arange(0.,360.,15.)
m.drawmeridians(meridians,labels=[0,0,0,1])
pl.title('div(V) at p = {0:.0f} hPa and $\Phi$ at p = {1:.0f} hPa'.format(p[n] / 100, p[n] / 100));
pl.tight_layout()

In [ ]:
n = get_pressure_index(50000.)

omega_scale = np.linspace(-1., 1., 21)
pl.figure()
m.contourf(lons, lats, omega[n,:,:], omega_scale,
           cmap=pl.cm.seismic, extend='both')
m.colorbar()

cz = m.contour(lons, lats, Phi[n,:,:], 15, colors='k', linewidths=1.5)
pl.clabel(cz, fontsize=10., fmt='%1.0f')

cp = m.contour(lons, lats, slp, 15, colors='k', linestyles=':', linewidths=1.)
pl.clabel(cp, fontsize=10., fmt='%1.0f')

m.drawcoastlines(color='#666666')
# draw parallels.
parallels = np.arange(-90.,90,15.)
m.drawparallels(parallels,labels=[1,0,0,0])
# draw meridians
meridians = np.arange(0.,360.,15.)
m.drawmeridians(meridians,labels=[0,0,0,1])
pl.title('$\omega$ at p = {0:.0f} hPa'.format(p[n] / 100));
pl.tight_layout()

In [ ]:
n = get_pressure_index(50000.)

pr_range = np.linspace(1., 8., 15)

pl.figure()
m.contourf(lons, lats, pr*1000, pr_range,
           cmap=pl.cm.Greens, extend='max')
m.colorbar()

cz = m.contour(lons, lats, Phi[n,:,:], 16, colors='k', linewidths=1.5)
pl.clabel(cz, fontsize=10., fmt='%1.0f')

omega_scale = np.arange(-2.1, -.2, 0.2)
co = m.contour(lons, lats, omega[n,:,:], omega_scale, colors='r', linestyles='-', linewidths=1.5)

cp = m.contour(lons, lats, slp, 16, colors='r', linestyles=':', linewidths=1.)
pl.clabel(cp, fontsize=10., fmt='%1.0f')

m.drawcoastlines(color='#666666')
# draw parallels.
parallels = np.arange(-90.,90,15.)
m.drawparallels(parallels,labels=[1,0,0,0])
# draw meridians
meridians = np.arange(0.,360.,15.)
m.drawmeridians(meridians,labels=[0,0,0,1])
pl.title('dzetadt_adv_f at p = {0:.0f} hPa'.format(p[n] / 100));
pl.tight_layout()

In [ ]:
nqv = 4
lat_plot = 48.
j_lat = abs(lat - lat_plot).argmin()

lonp, pp = np.meshgrid(lon, p)
pl.contourf(lon, p, T[:,j_lat,:], 21)
pl.colorbar()
pl.quiver(lonp[:,::nqv], pp[:,::nqv], u[:,j_lat,::nqv], -10*omega[:,j_lat,::nqv], 
          pivot='mid', width=1.2e-3, scale=1000)
pl.gca().invert_yaxis()
pl.xlim(lon_start, lon_end)
pl.ylim(100000, 10000);